# Libraries

In [1]:
import os
nnn = 1
os.environ["OMP_NUM_THREADS"] = str(nnn) # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = str(nnn) # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = str(nnn) # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = str(nnn) # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = str(nnn)  # export NUMEXPR_NUM_THREADS=1

In [2]:
from TELF.factorization.HNMFk import HNMFk
import pickle
import pandas as pd
import numpy as np
import scipy.sparse

In [3]:
import sys; sys.path.append("../../scripts/")
from generate_X import gen_data,gen_data_sparse

# Generate Synthetic Data

In [4]:
Xsp = gen_data_sparse(shape=[500, 500], density=0.01)["X"]
X = gen_data(R=4, shape=[500, 500])["X"]

/Users/maksim/Desktop/Code/T-ELF_public/examples/HNMFk/../../scripts/generate_X.py:60: UserWarning: Kruskal's theorem probably won't apply, may not have a unique nCPD.
  warnings.warn("Kruskal's theorem probably won't apply, may not have a unique nCPD.")


# Settings

In [5]:
Ks = np.arange(1, 10, 1)
perts = 2
iters = 5000
eps = 0.015
init = "nnsvd"
save_path = "HNMFk_results_path"
name = "example_HNMFk"

In [6]:
nmfk_params = {
    "n_perturbs":perts,
    "n_iters":iters,
    "epsilon":eps,
    "n_jobs":-1,
    "init":init, 
    "use_gpu":False,
    "save_path":save_path, 
    "predict_k_method":"sill",
    "verbose":False,
    "nmf_verbose":False,
    "transpose":False,
    "sill_thresh":0.8,
    "pruned":True,
    'nmf_method':'nmf_fro_mu',
    "calculate_error":False,
    "use_consensus_stopping":0,
    "calculate_pac":False,
    "consensus_mat":False,
    "perturb_type":"uniform",
    "perturb_multiprocessing":False,
    "perturb_verbose":False,
    "simple_plot":True
}

In [7]:
hnmfk_params = {
    # we can specify nmfk parameters for each depth, or use same for all depth
    # below will use the same nmfk parameters for all depths
    # when using for each depth, append to the list 
    # for example, [nmfk_params0, nmfk_params1, nmfk_params2] for depth of 2
    "nmfk_params": [nmfk_params], 
    # where to perform clustering, can be W or H
    # if W, row of X should be samples
    # if H, columns of X should be samples
    "cluster_on":"H",
    # how deep to go in each topic after root node
    # if -1, it goes until samples cannot be seperated further
    "depth":2,
    # stopping criteria for num of samples
    "sample_thresh":5,
    # if K2=True, decomposition is done only for k=2 instead of 
    # finding and predicting the number of stable latent features
    "K2":False,
    # after first nmfk, when selecting Ks search range, minimum k to start
    "Ks_deep_min":1,
    # After first nmfk, when selecting Ks search range, maximum k to try.
    # When None, maximum k will be same as k selected for parent node.
    "Ks_deep_max": 20,
    # after first nmfk, when selecting Ks search range, k step size
    "Ks_deep_step":1,
    # where to save
    "experiment_name":name
}

In [8]:
model = HNMFk(**hnmfk_params)
model.fit(X, Ks)

/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/numpy-1.26.2-py3.11-macosx-11.1-arm64.egg/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/numpy-1.26.2-py3.11-macosx-11.1-arm64.egg/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Example Traverse Graph

 Below functions can be utilized to walk the graph:

 ```python
model.traverse_nodes()
model.go_to_root()
model.get_node()
model.go_to_parent()
model.go_to_children(idx:int)
 ```

 We can reset the iterator to go back to the root node as shown below:

In [9]:
node = model.go_to_root()
node["name"]

'node0'

 HNMFk class includes a iterator that enables walking the graph nodes. Current node the iterator is at can be obtained as shown below (always starts at root node):

In [10]:
node = model.get_node()
node.keys()

dict_keys(['node_num', 'name', 'node_id', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data'])

 We can also see the name of the node:

In [11]:
node["name"]

'node0'

 And we can see the child nodes:

In [12]:
node["child_node_names"]

['node1', 'node6', 'node7', 'node12']

 We can go to the child node specified with an index. For example, to go to the first child, we index at 0. When we go to the child node, it will return the child node and set the iterator to the child node.

In [13]:
node = model.go_to_children(1)
node["name"]

'node6'

In [14]:
node.keys()

dict_keys(['node_num', 'name', 'node_id', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data'])

 Take a look at the parent node, which should be the root:

In [15]:
node["parent_node_name"]

'node0'

get_node() always returns the node we are currently at:

In [16]:
node = model.get_node()
node["name"]

'node6'

Go back to parent:

In [17]:
node = model.go_to_parent()
node["name"]

'node0'

 From each node, we can get the samples that was clustered in the node:

In [18]:
node["original_indices"][:5]

array([0, 1, 2, 3, 4])

We can also check if a given node in the graph a leaf node

In [19]:
node["leaf"]

False

 Finally, we can obtain all the nodes using the following method:

In [20]:
all_nodes = model.traverse_nodes()
len(all_nodes)

18

In [21]:
all_nodes[0].keys()

dict_keys(['node_num', 'name', 'node_id', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data'])